## Introduction

This is a textual write-up looking to explain the repo, the steps taken and the results achieved. No coding will be done here, but we will paste results and content from elsewhere in the repo.

It is hard to know the right amount of detail to go into. Commenting has also been done in the other notebooks and in the Python functions' files. But I may have explained something too much and something else too little.

## What this repo contains

This repo started as an exercise in learning PyTorch and understanding the attentional seq2seq architecture in full detail. 

We set up a toy French to English translation task, __implement the attentional seq2seq architecture of [Luong et al. (2015)](https://arxiv.org/pdf/1508.04025)__ in PyTorch, wrap it into a training loop and experiment with the effects of hyperparameters to find a good combination. Then the final model is investigated a bit.

I have borrowed from the [practical-pytorch](https://github.com/spro/practical-pytorch) repo by [spro](https://github.com/spro), in particular, [this notebook](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb). These resources provided the starting point and answered many of the early questions I had. However, all the code I have borrowed has been double-checked, sometimes fixed, sometimes reworked. And I have integrated this with my own code for training and plotting the learning curves, beam-search decoding, etc.

The particular translation task has been greatly simplified here for ease of compute, yet the same code should be able to handle much more involved seq2seq learning problems. All the computations here have been done using an Apple M2 Max machine, and the time taken, as reported in the legends of learning curves, is for that machine.

__Final result in a nutshell.__ Training on 5.2k French-English sentence pairs of length 2 to 5 tokens (inclusive), we are able to reach the cross-entropy score of 1.28:

<img src="results/5e-03_0.05_32_{'h_size':90,'dropout':0.2,'n_layers':2,'att_method':'general','c':'final_model'}.png" alt="pic" width="300"/>

We can then feed (tokenised) French into the model and use beam search to get its take on some probable translations. The figures in the picture below provide the log-likelihood:

<img src="write-up_pics/c.png" alt="pic" width="400"/>

Pretty happy with that!

## Sources

[The Luong paper](https://arxiv.org/pdf/1508.04025), one of the pioneering works on attention. We implement the architecture proposed therein, page 3 onwards.

The [paper](https://arxiv.org/pdf/1506.03099) proposing Scheduled Sampling. There is a set of helpful illustrations at the top of page 4. We implement Scheduled Sampling for the decoder we use.

[practical-pytorch](https://github.com/spro/practical-pytorch) repo provided the starting point for this work.

Oxford [Deep Learnig for NLP](https://github.com/oxford-cs-deepnlp-2017/lectures) lecture course. Good for understanding the broader research context at the time the attentional RNN models emerged.

## Data preparation

To obtain the dataset:
1. Go to this web page _http://www.manythings.org/anki/_
2. Download and extract the fra-eng.zip.
   
   <img src="write-up_pics/d.png" alt="pic" width="150"/>

3. Rename the file to "eng-fra.txt" (more appropriate).
4. Create a "datasets" folder in the repo and place the file there.
5. The "dataprep.ipynb" notebook should be end-to-end executable and should reproduce exactly upon re-run. 

We clean the file and simplify the corpus in terms of maximum length and number of words encountered. The vocabulary for both French and English ends up consisting of about 700 words each.

These are the files one ends up with in the "datasets" folder after executing the notebook:

<img src="write-up_pics/a.png" alt="pic" width="150"/>

Here trial1 and trial2 are just shortened datasets that can be useful for debugging and experiments.

## Coding up the model

The model we wish to implement is summarised in [the Luong paper](https://arxiv.org/pdf/1508.04025), page 3, as well as in many blogs

This has been coded up, in a batched form, in model_functions.py. We provide commenting throughout.

In terms of Scheduled Sampling, we implement a linear decay schedule, such that, from epoch 7 (inclusive) of the training process, only tokens sampled from the model itself (and not tokens of the translation, as provided in the training dataset) are fed into the model.

## Training and hyperparameter choices

Ok, so we have a model architecture coded up now. 

What are reasonable hyperparameter choices? Let's spend some time finding good hyperparameters, though we will not optimise for the very best possible choice here.

Start with a guess and see what happens:

<img src="results/5e-03_1_32_{'h_size':30,'dropout':0,'n_layers':2,'att_method':'dot','c':'first_v2'}.png" alt="pic" width="300"/>

We seem to be making reasonable choices, and the learning rate, in particular, looks sound. Also, our decay schedule for Scheduled Sampling looks reasonable, in that there is a smooth learning process. And at epoch 7 and onwards, when only the decoder's own predictions are fed into the next timestep of the decoder, there is no visible upset in the learing process.

Let's investigate what happens if we vary the number of layers and the hidden dimension:

<img src="results/atlases/hsize-layers.png" alt="pic" width="500"/>

It's not clear that having 3 layers makes a difference. However, increasing the dimensionality to 90 looks like a good choice. Let's now experiment with dropout and changing the attention mechanism. We will here disregard the 'concat' attention mechanism, because it is more computationally costly and also because it does not seem to improve performance significantly. The latter point is also reported in [the Luong paper](https://arxiv.org/pdf/1508.04025), page 8: "For *content-based* functions, our implementation *concat* does not yield good performances and more analysis should be done to understand the reason."

Let's have a look:

<img src="results/atlases/att_method-dropout.png" alt="pic" width="500"/>

Going from 'dot' to 'general' does seem to help. And increasing dropout to 0.2 looks sensible.

Finally, let's quickly check if we should increase or decrease the decoder's learning rate, relative to the encoder:

<img src="results/atlases/lr-ratio.png" alt="pic" width="500"/>

Increasing the learning rate of the decoder clearly does not make sense. However, we see better results by decreasing the decoder's learning rate, relative to the encoder. Perhaps this is to do with the fact that, in the backpropagation algorithm, the encoder sits further away from the output neurons than the decoder. 

Armed with these insights, we are able to reach the loss value of 1.28 (shown in Section 1 above) in a model that takes around 30min to train.

## Possible next steps

Here are some ideas: 
* Look a bit more into the embedding matrices we've learned, to see if interesting relationships emerge. Perhaps using tSNE or a clustering algorithm. 
* We could also use pre-trained embeddings, instead of training the embedding matrices from scratch.
* Plot and investigate attention patterns (as they do in [this notebook](https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb)).
* See what results are obtainable by non-attention seq2seq GRUs. Or try the transformer architecture.
* Implement the architecture extension mentioned in the [The Luong paper](https://arxiv.org/pdf/1508.04025), page 5:
  
  <img src="write-up_pics/b.png" alt="pic" width="250"/>
* Find a suitable model on HuggingFace to fine-tune, perhaps a T5.
* Change the dataset from a toy problem to something more challenging.
* Look at other data sets in the source web page _http://www.manythings.org/anki/_. Are some languages harder to learn than others? Could one train a multilingual toy translator?
* Keep the complexity of the current learning problem the same but double the data size and see what happens. One could possibly use an LLM and the present French-English dataset to synthetically generate a new dataset double the size.
* ...  